In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from dataclasses import dataclass
from typing import Optional, Dict, List, Tuple, Union
from pathlib import Path
from scipy import stats
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from palbociclib_signature_analysis import palbo_signatures
from palbo_RNAseq_analysis import palbo_RNAseq
from Integrative_analysis import geneset_heatmap, build_library

In [167]:
@dataclass
class DataPaths:
    """Centralize data paths configuration"""
    BASE_PATH: Path = Path('./input_data')
    CLINICAL_PATH: Path = BASE_PATH / 'clinical_features/brca_tcga_pan_can_atlas_2018_clinical_data.tsv'
    RECEPTOR_PATH: Path = BASE_PATH / 'clinical_features/TCGA_BRCA_clinical_receptors.txt'
    MRNA_BASE: Path = BASE_PATH / 'mRNA_data'

# split the data based molecular and clinical subtype

class TCGA_GeneExpressionData:
    """Handle gene expression data processing and analysis"""
    def __init__(self):
        self.clinical_feature = None
        self.receptor_feature = None
        self.gene_sets = {}
        
    def load_clinical_data(self, paths: DataPaths) -> None:
        """Load and process clinical data"""
        try:
            self.clinical_feature = pd.read_table(paths.CLINICAL_PATH, index_col=0)
            # NaN value as Unidentified 
            self.clinical_feature.fillna("Uni", inplace=True)
            self.receptor_feature = pd.read_table(paths.RECEPTOR_PATH, index_col=0)
            self._process_receptor_data()
        except Exception as e:
            logger.error(f"Error loading clinical data: {e}")
            raise
            
    def _process_receptor_data(self) -> None:
        """Process receptor status into clinical types of TNBC, Hormone Receptor(HR) positive and other"""
        conditions = [
            (self.receptor_feature['er_status_by_ihc'].eq('Negative') & 
             self.receptor_feature['pr_status_by_ihc'].eq('Negative') & 
             self.receptor_feature['her2_status_by_ihc'].eq('Negative')),
            ((self.receptor_feature['er_status_by_ihc'].eq('Positive') | 
              self.receptor_feature['pr_status_by_ihc'].eq('Positive')) & 
             self.receptor_feature['her2_status_by_ihc'].eq('Negative'))
        ]
        choices = ['TNBC', 'HR_positive']
        self.receptor_feature['clinical_types'] = np.select(conditions, choices, default='other')
    
    @staticmethod
    def preprocess_mRNA(fpath: Union[str, Path], remove_na: bool = True) -> pd.DataFrame:
        """Load and process cBioPortal data"""
        try:
            data = pd.read_table(fpath, index_col=1).iloc[:, 1:].T
            data.columns = [col[:12] for col in data.columns]  # sample to patient conversion
            return data.dropna(axis=1, how='all') if remove_na else data
        except Exception as e:
            logger.error(f"Error processing cBioPortal data from {fpath}: {e}")
            raise
            
    def load_gene_sets(self, paths: DataPaths) -> None:
        """Load all gene set expression data"""
        gene_sets = {
            'MYC_V1': 'MYC_V1/mRNA expression z-scores relative to diploid samples (RNA Seq V2 RSEM).txt',
            'G2M': 'G2M/mRNA expression z-scores relative to diploid samples (RNA Seq V2 RSEM).txt',
            'E2F': 'E2F/mRNA expression z-scores relative to diploid samples (RNA Seq V2 RSEM).txt',
            'EMT': 'EMT/mRNA expression z-scores relative to diploid samples (RNA Seq V2 RSEM).txt',
            'Glyco': 'Glycosis/mRNA expression z-scores relative to diploid samples (RNA Seq V2 RSEM).txt',
            'TNF_alpha': 'TNF_alpha/mRNA expression z-scores relative to diploid samples (RNA Seq V2 RSEM).txt'
        }
        
        for name, file_path in gene_sets.items():
            full_path = paths.MRNA_BASE / file_path
            self.gene_sets[name] = self.preprocess_mRNA(full_path)

    # merge two methods into one 
    def split_by_types(self, data: pd.DataFrame, by_type: Union[str, List[str]] = "clinical") -> Dict[str, pd.DataFrame]:
        """
        Split DataFrame by clinical or molecular subtypes.
        
        Args:
            data: DataFrame to split
            by_type: "clinical" for TNBC/HR classification or "molecular" for BRCA subtypes
            
        Returns:
            Dictionary of split DataFrames by subtype
        """
        TYPE_MAPPINGS = {
            "clinical": {
                "feature": "receptor_feature",
                "column": "clinical_types",
                "categories": ['TNBC', 'HR_positive', 'other']
            },
            "molecular": {
                "feature": "clinical_feature",
                "column": "Subtype",
                "categories": ['BRCA_LumA', 'BRCA_LumB', 'BRCA_Normal', 'BRCA_Basal', 'Uni']
            }
        }
        
        if by_type not in TYPE_MAPPINGS:
            raise ValueError(f"by_type must be one of {list(TYPE_MAPPINGS.keys())}")
        
        type_info = TYPE_MAPPINGS[by_type]
        feature_data = getattr(self, type_info["feature"])

    
        splits = {}
        for category in type_info["categories"]:
            mask = feature_data[type_info["column"]] == category
            patient_ids = feature_data[mask].index
            splits[category] = data.loc[:, data.columns.isin(patient_ids)]
        
        return splits
    
    def run_one_way_anova(self, data: pd.DataFrame, by_type: str = "clinical"):
        """
        Perform one-way ANOVA for each gene across groups
        
        Args:
            data: Gene expression matrix (genes × samples)
            by_type: Type of grouping to use ("clinical" or molecular)

        Returns:
            DataFrame with ANOVA results including F-statistic, p-value, and group means
        """
        if by_type == "clinical":
            groups = self.receptor_feature["clinical_types"]
        elif by_type == "molecular":
            groups = self.clinical_feature["Subtype"]
        else:
            raise ValueError("by_type must be either 'clinical' or 'molecular'")
        
        common_samples = groups.index.intersection(data.columns)
        groups = groups[common_samples]
        data = data[common_samples]
        results = []
        for gene in data.index:
            # Create lists of expression values for each group
            group_data = {group: data.loc[gene, groups == group] for group in groups.unique()}
            # Calculate group means
            group_means = {f"mean_{group}": values.mean() 
                          for group, values in group_data.items()}
            
            # Run ANOVA
            f_stat, p_val = stats.f_oneway(*group_data.values())
            
            # Add to results
            results.append({
                'gene': gene,
                'F_statistic': f_stat,
                'p_value': p_val,
                **group_means  # Include all group means in results
            })
        results_df = pd.DataFrame(results).set_index('gene')
        results_df = results_df.sort_values('p_value')
        
        # Add multiple testing correction
        results_df['adjusted_p_value'] = stats.false_discovery_control(results_df['p_value'])
        return results_df
    def analyze_all_genesets_anova(self, by_type: str = "clinical", output_dir: str = "./Processed_data_output") -> Dict[str, pd.DataFrame]:
        """
        Run ANOVA analysis for all gene sets and combine results.
        
        Args:
            by_type: Type of grouping to use ("clinical" or "molecular")
            output_dir: Directory to save results
            
        Returns:
            Dictionary containing ANOVA results for each gene set
        """
        if not self.gene_sets:
            raise ValueError("Gene sets not loaded. Please call load_gene_sets() first.")
        
        # Create output directory if it doesn't exist
        Path(output_dir).mkdir(parents=True, exist_ok=True)
        
        # Store results for each gene set
        all_results = {}
        
        # Combined significant results across gene sets
        significant_genes = pd.DataFrame()
        
        for name, data in self.gene_sets.items():
            logger.info(f"Running ANOVA analysis for {name} gene set")
            
            # Run ANOVA
            anova_results = self.run_one_way_anova(data, by_type)
            all_results[name] = anova_results
            
            # Extract significant genes (adjusted p-value < 0.05)
            sig_genes = anova_results[anova_results['adjusted_p_value'] < 0.05].copy()
            sig_genes['gene_set'] = name
            significant_genes = pd.concat([significant_genes, sig_genes])
            
            # Save individual results
            output_file = Path(output_dir) / f"{name}_anova_{by_type}.csv"
            anova_results.to_csv(output_file)
            
        # Save combined significant results
        significant_genes.sort_values('adjusted_p_value').to_csv(
            Path(output_dir) / f"all_significant_genes_anova_{by_type}.csv"
        )
        
        # Generate summary statistics
        summary_stats = pd.DataFrame({
            'gene_set': list(all_results.keys()),
            'total_genes': [len(df) for df in all_results.values()],
            'significant_genes': [len(df[df['adjusted_p_value'] < 0.05]) for df in all_results.values()],
            'min_p_value': [df['p_value'].min() for df in all_results.values()],
            'median_p_value': [df['p_value'].median() for df in all_results.values()]
        })
        
        # Save summary statistics
        summary_stats.to_csv(Path(output_dir) / f"anova_summary_stats_{by_type}.csv", index=False)
        
        return all_results

In [171]:
analysis = TCGA_GeneExpressionData()

In [177]:
paths = DataPaths()

In [181]:
analysis.load_clinical_data(paths)
analysis.load_gene_sets(paths)

In [183]:
analysis.analyze_all_genesets_anova(by_type = "clinical")

INFO:__main__:Running ANOVA analysis for MYC_V1 gene set
INFO:__main__:Running ANOVA analysis for G2M gene set
INFO:__main__:Running ANOVA analysis for E2F gene set
INFO:__main__:Running ANOVA analysis for EMT gene set
INFO:__main__:Running ANOVA analysis for Glyco gene set
INFO:__main__:Running ANOVA analysis for TNF_alpha gene set


{'MYC_V1':          F_statistic       p_value  mean_HR_positive  mean_other  mean_TNBC  \
 gene                                                                          
 SRPK1     136.069285  2.030414e-53         -0.149312    0.346196   2.182223   
 SERBP1    108.591382  1.084108e-43         -0.255612    0.001705   1.801383   
 MCM6       87.719680  5.056130e-36         -0.230935    0.117923   1.162317   
 MCM2       73.625411  1.067175e-30         -0.024261    0.352331   1.623878   
 HNRNPD     61.950509  3.406287e-26         -0.266674   -0.012696   0.958101   
 SRSF3      52.930935  1.181015e-22         -0.148428    0.154993   1.110799   
 STARD7     48.655684  5.875964e-21         -0.153500    0.068577   1.079349   
 ABCE1      33.944621  5.053916e-15         -0.210734    0.148332   0.881006   
 HNRNPA3    33.884466  5.348188e-15         -0.168143   -0.006883   0.798434   
 XPOT       27.053455  3.437015e-12          0.100036    0.304503   1.167577   
 LSM7       24.759076  3.06847

In [159]:
myc_data = analysis.gene_sets["MYC_V1"]
e2f_data = analysis.gene_sets["E2F"]

In [161]:
myc_data

,TCGA-3C-AAAU,TCGA-3C-AALI,TCGA-3C-AALJ,TCGA-3C-AALK,TCGA-4H-AAAK,TCGA-5L-AAT0,TCGA-5T-A9QA,TCGA-A1-A0SB,TCGA-A1-A0SD,TCGA-A1-A0SE,...,TCGA-UL-AAZ6,TCGA-UU-A93S,TCGA-V7-A7HQ,TCGA-W8-A86G,TCGA-WT-AB41,TCGA-WT-AB44,TCGA-XX-A899,TCGA-XX-A89A,TCGA-Z7-A8R5,TCGA-Z7-A8R6
DUT,0.6023,-0.1375,-0.6658,-0.3937,-0.0797,0.3578,0.4807,-0.3699,-0.8788,-0.2176,...,6.6143,1.1772,3.0750,-0.1701,1.0783,-0.9990,-0.2330,0.0574,1.2958,1.0998
STARD7,-1.3445,-0.2764,-0.5302,-1.2665,-0.9335,-0.2179,-0.7020,-0.0405,-0.2309,-0.0090,...,-1.6012,-0.3099,0.7151,-1.4520,0.6241,-1.6009,-1.2212,-1.2002,-0.4447,-0.1506
RPL22,-0.2799,-2.3845,-0.9937,-0.2862,0.4544,0.1556,-1.4079,1.9515,-1.1861,0.6681,...,-0.1155,-0.9998,2.1090,0.3729,0.2208,-0.0451,-0.2780,-1.2322,2.1107,-0.5633
CNBP,-1.6371,-0.9282,-1.4549,-0.9784,-0.1956,-0.3261,-0.1084,-0.0867,0.8095,-0.1692,...,-0.2118,-0.7244,-1.2668,-0.3096,-1.6335,0.9669,-0.8491,-1.0332,0.2624,-0.6953
SERBP1,-0.5621,-1.7568,-1.4712,-0.8030,-0.4045,-0.7926,-1.5519,1.7952,-1.0936,-0.0709,...,-1.6708,-1.1464,-0.9052,-0.9545,-1.5206,-1.6166,-0.7116,-0.6938,-0.5733,-0.1350
HNRNPD,-1.1776,-0.7836,0.4615,0.4492,0.1823,-0.1698,-1.2534,0.9026,-0.6136,-0.2878,...,-0.6070,0.9457,-0.5526,-0.3250,0.1636,-0.2629,-0.3807,-0.0410,0.3015,1.2330
TOMM70,0.8178,0.1936,-0.9403,-1.1175,-0.5851,-1.1624,-0.3755,0.0162,0.4686,0.0463,...,-0.9492,0.6148,-1.2486,-1.3549,-1.6473,-0.6400,-0.7334,-1.1909,-1.3480,-0.8959
SRPK1,0.5468,-0.5838,-1.0420,-0.7420,-0.5574,-1.1367,-0.8433,-0.0037,-0.5069,-0.2060,...,-0.6545,0.0083,-1.3060,-0.9467,-1.3627,-0.5393,-0.0709,-0.6847,-0.8775,-0.4622
SET,-1.5067,-0.3676,0.8911,-0.1952,0.5289,-0.2066,0.1951,0.6234,-0.0425,0.4370,...,-0.4211,-0.1343,-0.9303,-1.4724,-0.5626,-1.1949,-1.1709,-1.6092,-0.5946,2.2779
PCBP1,-2.0056,0.8832,2.1341,1.6845,-1.2200,0.7217,0.4968,-1.3006,-0.2146,0.4267,...,2.3847,0.2974,-1.4341,-0.1440,-0.8914,0.1954,-0.5682,-0.4654,-0.1856,2.0993


In [163]:
anova_clinical = analysis.run_one_way_anova(e2f_data, by_type="clinical")

In [165]:
anova_clinical

,F_statistic,p_value,mean_HR_positive,mean_other,mean_TNBC,adjusted_p_value
gene,,,,,,
LBR,131.763163,6.390630e-52,0.027616,0.178848,1.772061,1.853283e-50
CDC25A,122.263229,1.394945e-48,-0.211674,0.141532,1.455279,2.022671e-47
NASP,117.012891,1.024649e-46,-0.241282,0.145249,1.848403,9.904937e-46
MCM6,87.719680,5.056130e-36,-0.230935,0.117923,1.162317,3.665694e-35
MCM3,84.392979,8.905880e-35,-0.118023,0.416998,1.819874,5.165411e-34
MCM2,73.625411,1.067175e-30,-0.024261,0.352331,1.623878,5.158014e-30
ILF3,62.769078,1.635482e-26,-0.111279,0.081443,1.380250,6.775567e-26
HNRNPD,61.950509,3.406287e-26,-0.266674,-0.012696,0.958101,1.234779e-25
MTHFD2,50.725688,8.830184e-22,-0.089379,0.276946,1.173403,2.845281e-21
